In [0]:
# imports
import io
import numpy as np
import pandas as pd
import json

# Load Files

In [0]:
from google.colab import files
uploaded = files.upload()

Saving labeled.xlsx to labeled.xlsx


In [0]:
labeled = pd.read_excel(io.BytesIO(uploaded['labeled.xlsx']), encoding='latin-1')

In [0]:
from google.colab import files
uploaded = files.upload()

Saving unlabeled.xlsx to unlabeled.xlsx


In [0]:
unlabeled = pd.read_excel(io.BytesIO(uploaded['unlabeled.xlsx']), encoding='latin-1') # subset of 5000 unlabeled records

In [0]:
from google.colab import files
uploaded = files.upload()

Saving predictive_text.json to predictive_text.json


In [0]:
predictive_text = json.loads(uploaded["predictive_text.json"].decode("utf-8"))['predictive_text']

# Create Columns For Labeled Data

In [0]:
labeled.head()

,AgeGroup,Sex,Pregnancy,Query Text,Result,Specialty
0,4,male,not pregnant,severe leg pain|vomiting|fever|mottled skin,Sepsis and Shock,INFECT
1,8,male,not pregnant,headache|word finding difficulties|parietial a...,Brain Neoplasms,NEOPL
2,7,female,not pregnant,breathing problem|migraine|weight loss|arm and...,Lung Neoplasms,NEOPL
3,6,female,not pregnant,leg cut|intense pain|vomiting,Necrotising Fasciitis,INFECT
4,8,female,not pregnant,headache|blurred vision,Giant Cell Arteritis,RHEUM


In [0]:
# loop through each row in the labeled
# for each symptom in query text input, if that symptom is a predictive text symptom (not free text input), create a column and fill in 1
for index, row in labeled.iterrows():
  symptoms = row['Query Text'].split('|')
  for symptom in symptoms:
    symptom = symptom.strip()
    if symptom in predictive_text and symptom not in labeled.columns: # columns now contain all predictive text symptoms
      labeled[symptom] = 0
      labeled.at[index, symptom] = 1
    elif symptom in predictive_text and symptom in labeled.columns:
      labeled.at[index, symptom] = 1
    else:
      continue

### this creates labeled data frame with columns for only features found in labeled data


In [0]:
labeled.head()

,AgeGroup,Sex,Pregnancy,Query Text,Result,Specialty,vomiting,fever,mottled skin,headache,word finding difficulties,breathing problem,migraine,weight loss,leg cut,blurred vision,dizziness,fainting,smoker,lethargy,irritability,fatigue,swollen gums,increased heart rate,chickenpox,sweats,abdominal pain,apprehension,hypertension,swallowing problem,coughing,chest pain resting,nausea,stiff neck,malaise,sob,hemoptysis,chronic cough,coughing up blood,proteinuria,...,lightheaded,low blood pressure,low sodium,low cortisol,non pruritic rash,positive antinuclear antibodies,sound sensitivity,unable to speak,hematochezia,tachypneic,decreased urine output,african american,low bp,bph,high lipids,unable to maintain an erection,pneumothorax,hypermobility,absent pulse,melena,renal insufficiency,family history of sudden cardiac death,anemic,vitamin d deficiency,lower abdominal discomfort,constant headache,hoarse voice,shallow breathing,jaw ache,face ache,ear pain,low rbc count,elevated monocytes,overweight,thirsty,unable to eat,irritable bowel,decreased bp,post partum,neutropenic
0,4,male,not pregnant,severe leg pain|vomiting|fever|mottled skin,Sepsis and Shock,INFECT,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8,male,not pregnant,headache|word finding difficulties|parietial a...,Brain Neoplasms,NEOPL,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,female,not pregnant,breathing problem|migraine|weight loss|arm and...,Lung Neoplasms,NEOPL,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6,female,not pregnant,leg cut|intense pain|vomiting,Necrotising Fasciitis,INFECT,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,female,not pregnant,headache|blurred vision,Giant Cell Arteritis,RHEUM,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# get dummies for categorical variables
labeled_with_dummies = pd.get_dummies(labeled, columns=['Sex', 'Pregnancy'])

In [0]:
labeled_with_dummies.head()

,AgeGroup,Query Text,Result,Specialty,vomiting,fever,mottled skin,headache,word finding difficulties,breathing problem,migraine,weight loss,leg cut,blurred vision,dizziness,fainting,smoker,lethargy,irritability,fatigue,swollen gums,increased heart rate,chickenpox,sweats,abdominal pain,apprehension,hypertension,swallowing problem,coughing,chest pain resting,nausea,stiff neck,malaise,sob,hemoptysis,chronic cough,coughing up blood,proteinuria,hematuria,sore throat,...,non pruritic rash,positive antinuclear antibodies,sound sensitivity,unable to speak,hematochezia,tachypneic,decreased urine output,african american,low bp,bph,high lipids,unable to maintain an erection,pneumothorax,hypermobility,absent pulse,melena,renal insufficiency,family history of sudden cardiac death,anemic,vitamin d deficiency,lower abdominal discomfort,constant headache,hoarse voice,shallow breathing,jaw ache,face ache,ear pain,low rbc count,elevated monocytes,overweight,thirsty,unable to eat,irritable bowel,decreased bp,post partum,neutropenic,Sex_female,Sex_male,Pregnancy_not pregnant,Pregnancy_pregnant
0,4,severe leg pain|vomiting|fever|mottled skin,Sepsis and Shock,INFECT,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,8,headache|word finding difficulties|parietial a...,Brain Neoplasms,NEOPL,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,7,breathing problem|migraine|weight loss|arm and...,Lung Neoplasms,NEOPL,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,6,leg cut|intense pain|vomiting,Necrotising Fasciitis,INFECT,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
4,8,headache|blurred vision,Giant Cell Arteritis,RHEUM,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


In [0]:
labeled_with_dummies.to_excel('labeled_with_dummies.xlsx')

# Create Columns For Unlabeled Data

In [0]:
unlabeled.head()

,AgeGroup,Sex,Pregnancy,Query Text,Result,Specialty
0,7,female,not pregnant,upper back skin sensation | nauseous | gipping...,"Sarcoidosis, Heart Attack, Pyelonephritis, Lym...",General
1,8,male,not pregnant,nausea | vomiting | spine pain | strange sensa...,"Sarcoidosis, Heart Attack, Abdominal Aortic An...",General
2,9,male,not pregnant,ringing tinnitus | ear pressure | abnormal hea...,"Ear Infection, Meniere's Disease, Superior Can...",General
3,6,female,not pregnant,stomach pain after eating | stomach cramps | s...,"Intestinal Ischemia , Irritable Bowel Syndrome...",General
4,9,male,not pregnant,ringing tinnitus | ear pressure | abnormal hea...,"Meniere's Disease, Ear Infection, Superior Can...",General


In [0]:
# create a column for each of the symptom features in the labeled data so they are the same diemnsions
for col in labeled.columns:
  if col not in unlabeled.columns:
    unlabeled[col] = 0

In [0]:
# loop through each row in the unlabeled data
# for each symptom in query text input, if that symptom is a feature, fill in 1 
for index, row in unlabeled.iterrows():
  symptoms = row['Query Text'].split('|')
  for symptom in symptoms:
    symptom = symptom.strip()
    if symptom in unlabeled.columns:
      unlabeled.at[index, symptom] = 1
    else:
      continue

In [0]:
# get dummies for categorical
unlabeled_with_dummies = pd.get_dummies(unlabeled, columns=['Sex', 'Pregnancy'])

In [0]:
unlabeled_with_dummies.head()

,AgeGroup,Query Text,Result,Specialty,vomiting,fever,mottled skin,headache,word finding difficulties,breathing problem,migraine,weight loss,leg cut,blurred vision,dizziness,fainting,smoker,lethargy,irritability,fatigue,swollen gums,increased heart rate,chickenpox,sweats,abdominal pain,apprehension,hypertension,swallowing problem,coughing,chest pain resting,nausea,stiff neck,malaise,sob,hemoptysis,chronic cough,coughing up blood,proteinuria,hematuria,sore throat,...,non pruritic rash,positive antinuclear antibodies,sound sensitivity,unable to speak,hematochezia,tachypneic,decreased urine output,african american,low bp,bph,high lipids,unable to maintain an erection,pneumothorax,hypermobility,absent pulse,melena,renal insufficiency,family history of sudden cardiac death,anemic,vitamin d deficiency,lower abdominal discomfort,constant headache,hoarse voice,shallow breathing,jaw ache,face ache,ear pain,low rbc count,elevated monocytes,overweight,thirsty,unable to eat,irritable bowel,decreased bp,post partum,neutropenic,Sex_female,Sex_male,Pregnancy_not pregnant,Pregnancy_pregnant
0,7,upper back skin sensation | nauseous | gipping...,"Sarcoidosis, Heart Attack, Pyelonephritis, Lym...",General,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,8,nausea | vomiting | spine pain | strange sensa...,"Sarcoidosis, Heart Attack, Abdominal Aortic An...",General,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,9,ringing tinnitus | ear pressure | abnormal hea...,"Ear Infection, Meniere's Disease, Superior Can...",General,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,6,stomach pain after eating | stomach cramps | s...,"Intestinal Ischemia , Irritable Bowel Syndrome...",General,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
4,9,ringing tinnitus | ear pressure | abnormal hea...,"Meniere's Disease, Ear Infection, Superior Can...",General,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [0]:
unlabeled_with_dummies.to_excel('unlabeled_with_dummies.xlsx')

# Baselines

In [0]:
from google.colab import files
uploaded = files.upload()

Saving labeled_with_dummies.xlsx to labeled_with_dummies.xlsx


In [0]:
labeled_w_dummies = pd.read_excel(io.BytesIO(uploaded['labeled_with_dummies.xlsx']), encoding='latin-1')

In [0]:
labeled_w_dummies.head()

,Unnamed: 0,AgeGroup,Query Text,Result,Specialty,vomiting,fever,mottled skin,headache,word finding difficulties,breathing problem,migraine,weight loss,leg cut,blurred vision,dizziness,fainting,smoker,lethargy,irritability,fatigue,swollen gums,increased heart rate,chickenpox,sweats,abdominal pain,apprehension,hypertension,swallowing problem,coughing,chest pain resting,nausea,stiff neck,malaise,sob,hemoptysis,chronic cough,coughing up blood,proteinuria,hematuria,...,non pruritic rash,positive antinuclear antibodies,sound sensitivity,unable to speak,hematochezia,tachypneic,decreased urine output,african american,low bp,bph,high lipids,unable to maintain an erection,pneumothorax,hypermobility,absent pulse,melena,renal insufficiency,family history of sudden cardiac death,anemic,vitamin d deficiency,lower abdominal discomfort,constant headache,hoarse voice,shallow breathing,jaw ache,face ache,ear pain,low rbc count,elevated monocytes,overweight,thirsty,unable to eat,irritable bowel,decreased bp,post partum,neutropenic,Sex_female,Sex_male,Pregnancy_not pregnant,Pregnancy_pregnant
0,0,4,severe leg pain|vomiting|fever|mottled skin,Sepsis and Shock,INFECT,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,1,8,headache|word finding difficulties|parietial a...,Brain Neoplasms,NEOPL,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,2,7,breathing problem|migraine|weight loss|arm and...,Lung Neoplasms,NEOPL,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,3,6,leg cut|intense pain|vomiting,Necrotising Fasciitis,INFECT,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
4,4,8,headache|blurred vision,Giant Cell Arteritis,RHEUM,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


In [0]:
labeled_w_dummies.columns

Index(['Unnamed: 0', 'AgeGroup', 'Query Text', 'Result', 'Specialty',
       'vomiting', 'fever', 'mottled skin', 'headache',
       'word finding difficulties',
       ...
       'thirsty', 'unable to eat', 'irritable bowel', 'decreased bp',
       'post partum', 'neutropenic', 'Sex_female', 'Sex_male',
       'Pregnancy_not pregnant', 'Pregnancy_pregnant'],
      dtype='object', length=832)

In [0]:
from sklearn.model_selection import train_test_split

X = labeled_w_dummies.drop(columns = ['Unnamed: 0', 'Query Text', 'Result', 'Specialty'])
y = labeled_w_dummies['Specialty']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [0]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(max_depth=2, random_state=0)
clf_rf.fit(X_train, y_train)
score = clf_rf.score(X_test, y_test)
print(score)

0.2553191489361702


In [0]:
import sklearn.metrics as metrics
pred_rf = clf_rf.predict(X_test)

print(metrics.classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

      ALLERG       0.00      0.00      0.00         1
      CARDIO       0.00      0.00      0.00         3
        DERM       0.00      0.00      0.00         5
        ENDO       0.00      0.00      0.00        11
      GASTRO       0.00      0.00      0.00        10
        GENE       0.00      0.00      0.00         1
       HEMAT       0.00      0.00      0.00         2
      HEPATO       0.00      0.00      0.00         1
      INFECT       0.26      1.00      0.41        36
       METAB       0.00      0.00      0.00         1
       NEOPL       0.00      0.00      0.00        10
      NEPHRO       0.00      0.00      0.00         2
       NEURO       0.00      0.00      0.00        18
       OBGYN       0.00      0.00      0.00         3
       ORTHO       0.00      0.00      0.00         4
       PSYCH       0.00      0.00      0.00         6
        RESP       0.00      0.00      0.00         6
       RHEUM       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
from sklearn.neighbors import KNeighborsClassifier as KNN
clf_knn = KNN()
clf_knn.fit(X_train, y_train)
score = clf_knn.score(X_test, y_test)
print(score)

0.09219858156028368


In [0]:
pred_knn = clf_knn.predict(X_test)

print(metrics.classification_report(y_test, pred_knn))

              precision    recall  f1-score   support

      ALLERG       0.00      0.00      0.00         1
      CARDIO       0.00      0.00      0.00         3
    CARDIO         0.00      0.00      0.00         0
        DERM       0.00      0.00      0.00         5
         EAR       0.00      0.00      0.00         0
        ENDO       0.00      0.00      0.00        11
      GASTRO       0.00      0.00      0.00        10
        GENE       0.00      0.00      0.00         1
       HEMAT       0.00      0.00      0.00         2
      HEPATO       0.00      0.00      0.00         1
       IMMUN       0.00      0.00      0.00         0
      INFECT       0.50      0.17      0.25        36
       METAB       0.00      0.00      0.00         1
       NEOPL       0.00      0.00      0.00        10
      NEPHRO       0.00      0.00      0.00         2
       NEURO       0.16      0.28      0.20        18
       OBGYN       0.00      0.00      0.00         3
       ORTHO       1.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
from sklearn.linear_model import RidgeClassifier
clf_r = RidgeClassifier()
clf_r.fit(X_train, y_train)
score = clf_r.score(X_test, y_test)
print(score)

0.3049645390070922


In [0]:
pred_r = clf_r.predict(X_test)

print(metrics.classification_report(y_test, pred_r))

              precision    recall  f1-score   support

      ALLERG       0.00      0.00      0.00         1
      CARDIO       0.00      0.00      0.00         3
    CARDIO         0.00      0.00      0.00         0
        DERM       0.00      0.00      0.00         5
        ENDO       0.00      0.00      0.00        11
      GASTRO       0.35      0.60      0.44        10
        GENE       0.00      0.00      0.00         1
       HEMAT       0.00      0.00      0.00         2
      HEPATO       0.00      0.00      0.00         1
      INFECT       0.46      0.61      0.52        36
       METAB       0.00      0.00      0.00         1
       NEOPL       0.20      0.20      0.20        10
      NEPHRO       0.00      0.00      0.00         2
       NEURO       0.18      0.33      0.23        18
       OBGYN       0.00      0.00      0.00         3
       ORTHO       1.00      0.50      0.67         4
       PSYCH       0.75      0.50      0.60         6
        RESP       0.25    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
